# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f02dccb8eb0>
├── 'a' --> tensor([[ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555]])
└── 'x' --> <FastTreeValue 0x7f02dccb8f70>
    └── 'c' --> tensor([[-0.0185,  0.1023, -0.2857,  0.6214],
                        [ 0.4965, -0.6195, -0.7838, -0.4765],
                        [-1.2410, -1.4649, -0.2316, -1.2231]])

In [4]:
t.a

tensor([[ 0.0611,  0.3375,  0.6706],
        [-0.9574, -2.1367,  0.5555]])

In [5]:
%timeit t.a

62.9 ns ± 0.0297 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f02dccb8eb0>
├── 'a' --> tensor([[ 0.6108, -1.2709, -1.1665],
│                   [ 0.2176, -0.8124,  1.0967]])
└── 'x' --> <FastTreeValue 0x7f02dccb8f70>
    └── 'c' --> tensor([[-0.0185,  0.1023, -0.2857,  0.6214],
                        [ 0.4965, -0.6195, -0.7838, -0.4765],
                        [-1.2410, -1.4649, -0.2316, -1.2231]])

In [7]:
%timeit t.a = new_value

59.5 ns ± 0.0583 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555]]),
    x: Batch(
           c: tensor([[-0.0185,  0.1023, -0.2857,  0.6214],
                      [ 0.4965, -0.6195, -0.7838, -0.4765],
                      [-1.2410, -1.4649, -0.2316, -1.2231]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0611,  0.3375,  0.6706],
        [-0.9574, -2.1367,  0.5555]])

In [11]:
%timeit b.a

55.6 ns ± 0.0669 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5206, -0.6563,  0.8132],
               [-3.1887, -0.9167,  0.5971]]),
    x: Batch(
           c: tensor([[-0.0185,  0.1023, -0.2857,  0.6214],
                      [ 0.4965, -0.6195, -0.7838, -0.4765],
                      [-1.2410, -1.4649, -0.2316, -1.2231]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.0651 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

818 ns ± 0.082 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 63.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 595 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f02e0ce90d0>
├── 'a' --> tensor([[[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]],
│           
│                   [[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]],
│           
│                   [[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]],
│           
│                   [[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]],
│           
│                   [[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]],
│           
│                   [[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]],
│           
│                   [[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]],
│           
│                   [[ 0.0611,  0.3375,  0.6706],
│                    [-0.9574, -2.1367,  0.5555]]])
└── 'x' --> <FastTreeValue 0x7f02e0ce9400>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.7 µs ± 25.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f02084b0f40>
├── 'a' --> tensor([[ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555],
│                   [ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555],
│                   [ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555],
│                   [ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555],
│                   [ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555],
│                   [ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555],
│                   [ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555],
│                   [ 0.0611,  0.3375,  0.6706],
│                   [-0.9574, -2.1367,  0.5555]])
└── 'x' --> <FastTreeValue 0x7f02084f89d0>
    └── 'c' --> tensor([[-0.0185,  0.1023, -0.2857,  0.6214],
                        [ 0.4965, -0.6195, -0.7838, -0.4765],
                 

In [23]:
%timeit t_cat(trees)

31.3 µs ± 35.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.4 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]],
       
               [[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]],
       
               [[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]],
       
               [[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]],
       
               [[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]],
       
               [[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]],
       
               [[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]],
       
               [[ 0.0611,  0.3375,  0.6706],
                [-0.9574, -2.1367,  0.5555]]]),
    x: Batch(
           c: tensor([[[-0.0185,  0.1023, -0.2857,  0.6214],
                       [ 0.4965, -0.6195, -0.7838, -0.4765],
                       [-1.2410, -1.4649, -0.2316, -1.2231]],
         

In [26]:
%timeit Batch.stack(batches)

79.7 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555],
               [ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555],
               [ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555],
               [ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555],
               [ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555],
               [ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555],
               [ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555],
               [ 0.0611,  0.3375,  0.6706],
               [-0.9574, -2.1367,  0.5555]]),
    x: Batch(
           c: tensor([[-0.0185,  0.1023, -0.2857,  0.6214],
                      [ 0.4965, -0.6195, -0.7838, -0.4765],
                      [-1.2410, -1.4649, -0.2316, -1.2231],
                      [-0.0185,  0.1023, -0.2857,  0.6214],
                      [ 0.4965, -

In [28]:
%timeit Batch.cat(batches)

147 µs ± 369 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
